### 카테고리별 지원 파싱 (임신보육지원)
임신보육지원 (96~135페이지) 파싱

In [1]:
import os
import json
import time
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
import fitz

load_dotenv()

True

In [2]:
class TableData(BaseModel):
    columns: list = Field(description="표의 열 제목들")
    rows: list = Field(description="표의 행 데이터들")

class WelfareProgram(BaseModel):
    title: str = Field(description="복지 프로그램의 정확한 지원명")
    target: str = Field(description="지원 대상자")
    content: str = Field(description="지원 내용")
    method: str = Field(description="신청 방법")
    inquiry: str = Field(description="문의처")
    tables: list[TableData] = Field(description="표 데이터 구조")

In [3]:
def extract_page_text(pdf_path, page_num):
    doc = fitz.open(pdf_path)
    page = doc.load_page(page_num - 1)
    text = page.get_text("text")
    doc.close()
    return text

In [4]:
def parse_pregnancy_support(text, page_num):
    llm = ChatOpenAI(
        model="gpt-3.5-turbo",
        temperature=0,
        request_timeout=60,
        max_retries=3
    )
    
    parser = JsonOutputParser(pydantic_object=WelfareProgram)
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", """임신보육지원 문서 전용 파서입니다.

검증 로직:
1. "대상", "내용", "방법", "문의" 4개 필수 키워드 존재 확인
2. 키워드 중 하나라도 없으면 null 반환

제도명 추출 규칙:
- "나에게 힘이 되는 복지서비스"는 페이지 헤더이므로 제외
- "임신·보육 지원"은 카테고리명이므로 제외
- 실제 제도명만 추출 (예: "아이돌봄서비스", "첫만남이용권", "육아휴직급여")

표 처리 규칙:
1. content: 표 내용을 자연어로 요약하여 포함
2. tables: 표가 있으면 구조화된 데이터로 저장
   - columns: 실제 표의 열 제목을 정확히 추출 (예: ["서비스유형", "이용시간", "정부지원금"])
   - rows: 실제 표의 행 데이터를 정확히 추출 (예: [["영아종일제", "월 200시간", "시간당 12,000원"]])
   - 표의 실제 구조를 그대로 반영하여 저장
3. 표가 없으면 tables는 빈 배열

추출 최적화:
- 제도명: 실제 임신보육지원제도명만
- 대상: 임신여부, 자녀연령, 소득기준 등 구체적 조건
- 내용: 지원금액, 지원기간, 지원내용 (표 데이터 요약 포함)
- 방법: 신청기관, 온라인/오프라인 구분, 필요서류
- 문의: 콜센터, 웹사이트, 담당기관
- tables: 표가 있으면 실제 컬럼명과 데이터로 구조화하여 저장

{format_instructions}"""),

        ("human", """임신보육지원 문서를 분석합니다.

1단계: 필수 키워드 확인
- "대상", "내용", "방법", "문의" 4개 키워드 모두 존재하는지 확인
- 하나라도 없으면 null 반환

2단계: 제도명 추출
- "나에게 힘이 되는 복지서비스" 제외
- "임신·보육 지원" 제외
- 실제 임신보육지원제도명만 추출

3단계: 정보 추출
- content: 표 내용을 자연어로 요약하여 포함
- tables: 표가 있으면 실제 컬럼명과 행 데이터를 정확히 추출하여 저장
- 각 키워드 바로 다음 내용을 정확히 추출

텍스트:
{text}
""")
    ])
    
    chain = prompt | llm | parser
    
    for attempt in range(3):
        try:
            result = chain.invoke({
                "text": text,
                "format_instructions": parser.get_format_instructions()
            })
            
            if result is None:
                print(f"페이지 {page_num}: 필수 키워드 부족으로 건너뛰기")
                return None
            
            result["page"] = page_num
            result["category"] = "임신보육지원"
            
            return result
        except Exception as e:
            print(f"파싱 시도 {attempt + 1}/3 실패 (페이지 {page_num}): {e}")
            if attempt < 2:
                time.sleep(2)
            else:
                return None

In [5]:
# 실행
pdf_path = "../../data/pdfs/[통합]+2025+나에게+힘이+되는+복지서비스.pdf"
target_pages = list(range(96, 136))  # 임신보육지원 섹션 (96~135페이지)
pregnancy_programs = []

for i, page_num in enumerate(target_pages):
    print(f"페이지 {page_num} 파싱 중... ({i+1}/{len(target_pages)})")
    
    page_text = extract_page_text(pdf_path, page_num)
    program = parse_pregnancy_support(page_text, page_num)
    
    if program:
        pregnancy_programs.append(program)
        table_count = len(program.get('tables', []))
        print(f"{program['title']} 추출 완료 (표 {table_count}개)")
    else:
        print(f"페이지 {page_num} 건너뛰기")
    
    if i < len(target_pages) - 1:
        time.sleep(1)

페이지 96 파싱 중... (1/40)
임신 사전건강관리 지원 추출 완료 (표 0개)
페이지 97 파싱 중... (2/40)
난임부부 시술비 지원 추출 완료 (표 1개)
페이지 98 파싱 중... (3/40)
난임·임산부 심리상담서비스 추출 완료 (표 1개)
페이지 99 파싱 중... (4/40)
냉동난자 사용 보조생식술 지원 추출 완료 (표 0개)
페이지 100 파싱 중... (5/40)
표준모자보건수첩 제공 추출 완료 (표 0개)
페이지 101 파싱 중... (6/40)
임산부 철분제·엽산제 지원 추출 완료 (표 0개)
페이지 102 파싱 중... (7/40)
임신·출산 진료비 추출 완료 (표 0개)
페이지 103 파싱 중... (8/40)
요양비(출산비)지원 추출 완료 (표 0개)
페이지 104 파싱 중... (9/40)
청소년산모 임신·출산 의료비 지원 추출 완료 (표 0개)
페이지 105 파싱 중... (10/40)
위기임신 및 보호출산 지원 추출 완료 (표 1개)
페이지 106 파싱 중... (11/40)
국민행복카드 임신·출산 진료비 지원 추출 완료 (표 0개)
페이지 107 파싱 중... (12/40)
첫만남이용권 추출 완료 (표 0개)
페이지 108 파싱 중... (13/40)
산모·신생아 건강관리 지원사업 추출 완료 (표 1개)
페이지 109 파싱 중... (14/40)
우체국 대한민국 엄마보험(무료 공익보험) 추출 완료 (표 0개)
페이지 110 파싱 중... (15/40)
국민연금 출산크레딧 추출 완료 (표 1개)
페이지 111 파싱 중... (16/40)
임신보육지원 추출 완료 (표 1개)
페이지 112 파싱 중... (17/40)
출산급여 및 출산휴가급여 지원 추출 완료 (표 1개)
페이지 113 파싱 중... (18/40)
육아휴직급여지원 추출 완료 (표 2개)
페이지 114 파싱 중... (19/40)
출산육아기 고용안정장려금(대체인력 지원금) 추출 완료 (표 0개)
페이지 115 파싱 중... (20/40)
출산육아기 고용안정장려

In [6]:
# 결과 저장
os.makedirs("outputs", exist_ok=True)
output_path = "../../data/outputs/임신보육지원_2025.json"

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(pregnancy_programs, f, ensure_ascii=False, indent=2)

print(f"\n파싱 완료: {len(pregnancy_programs)}개 프로그램")
print(f"저장 위치: {output_path}")


파싱 완료: 40개 프로그램
저장 위치: ../../data/outputs/임신보육지원_2025.json


In [7]:
# 결과 미리보기
for program in pregnancy_programs:
    table_count = len(program.get('tables', []))
    print(f"\n {program['title']} (페이지 {program['page']}, 표 {table_count}개)")
    target = program.get('target')
    if isinstance(target, str) and target:
        print(f"대상: {target[:80]}...")
    else:
        print("대상: 정보 없음")
    print(f"내용: {program['content'][:80]}...")
    if table_count > 0:
        print(f"표: {program['tables'][0]['columns'][:3]}...")


 임신 사전건강관리 지원 (페이지 96, 표 0개)
대상: 모든 20~49세 남녀 중 가임력 검사 희망자...
내용: 임신 사전건강관리 지원은 20~49세 남녀 중 가임력 검사 희망자를 대상으로 합니다. 주요 내용은 주기별 최대 3회의 검사를 통한 비용 지원이며...

 난임부부 시술비 지원 (페이지 97, 표 1개)
대상: 난임진단을 받은 난임부부...
내용: 건강보험 급여가 적용된 체외수정(신선･동결배아), 인공수정 시술비 중 일부·전액본인부담금 90% 지원. 출산당 체외수정 최대 20회, 인공수정 ...
표: ['구분', '지원 금액']...

 난임·임산부 심리상담서비스 (페이지 98, 표 1개)
대상: 난임 및 산전·후 우울증을 겪는 부부 등...
내용: 난임 및 산전·후 우울증을 겪는 부부 등을 위한 심리상담 서비스를 제공합니다. 2025년부터 전북(예수병원), 경남(창원한마음병원)에 난임·임산...
표: ['구분', '운영기관', '주소']...

 냉동난자 사용 보조생식술 지원 (페이지 99, 표 0개)
대상: 1. 냉동한 난자를 사용하여 임신･출산을 시도하는 부부(난임부부 포함)
2. 신청일 기준 1년 이상 사실상 혼인관계를 유지하였다고 관할 보건소로...
내용: 냉동난자 해동, 체외수정 신선배아 시술비를 지원하며, 부부당 최대 2회까지 지원하고 1회당 최대 100만 원을 지원합니다. 사전 신청 없이 냉동...

 표준모자보건수첩 제공 (페이지 100, 표 0개)
대상: 임신부 또는 출생사실이 확인된 영유아...
내용: 임신준비부터 자녀육아까지 유용한 정보가 수록된 수첩을 배부하며, 산전검진, 예방접종 등 진료일정관리 및 건강, 육아 정보를 제공합니다....

 임산부 철분제·엽산제 지원 (페이지 101, 표 0개)
대상: 임신부...
내용: 임신부를 대상으로 철분제와 엽산제를 지원합니다. 철분제는 임신 16주부터 5개월분까지, 엽산제는 임신 전후 3개월분까지 지원됩니다. 이는 철분결...

 임신·출산 진료비 (페이지 102, 표 0개)
대